Here we will implement the k-Nearest Neighbors (KNN) algorithm from scratch using Python! 

KNN is a **supervised** classification and regression method. This differs from **k-means clustering** in that **k-means clustering** is an **unsupervised** method for clustering unlabeled data. 

We will work with the [Iris datsaet from sklearn](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html).

In [55]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from collections import Counter

Let's begin by loading the dataset from sklearn and converting it to a data frame for easy viewing.

In [4]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


Because the KNN algorithm is a supervised learning method, it relies on labeled data. It essentially clasifies new data points based on knowledge of the $k$ nearest, or most similar, data points which already have known categorical labels. Thus, important steps in this implementation of KNN are:

1. Split dataset into known (train) data points and unknown (test) data points to label with the algorithm. 
2. Determine a value of $k$ 
3. Define a function for calculating distance/similarity between data points 

We will use Euclidean distance in this tutorial to calculate distance between data points. Specifically, for each unknown (test) data point in the data, we will:
1. Find the Euclidian distance to each training data point
2. Store the distances and label of the training data point on an ordered list and sort
3. Choose the top $k$ entries on this list
4. Determine the most frequently appearing label of the top $k$ entries and assign this label to the test data point

Let's start by defining the distance function here:

In [38]:
def euclidean_distance(test_point, train_point):
    assert len(test_point)==len(train_point), "Data points do not have the same dimensions"
    ssd = 0 # sum of squared distances
    for i in range(len(test_point)):
        ssd += (test_point[i]-train_point[i])**2
    return ssd**(0.5)

We will split up the iris dataset into a training dataset and a test dataset. We will compute distances between each test data point and all of the training data points to make predictions about the test data points, and then measure the accuracy of these predictions. By default, the [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function from sklearn sets aside 25% of the dataset for the test set and 75% for the training set.

In [86]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target)

print('%d points in training dataset, %d points in test set' % (len(X_train), len(X_test)))

112 points in training dataset, 38 points in test set


We will define some helper functions and a function to run the algorithm.

In [77]:
def sort_distances(train, test):
    '''
    for a given test point, find distance to all points in training set
    '''
    distances = []
    for i in range(len(train)):
        distances.append(euclidean_distance(test, train[i,]))
    return(distances)

def predict_class(distances, y_train, k):
    '''
    given distances, labels of training data, and k
    identify most common label among k nearest neighbors
    '''
    neighbor_labels = y_train[np.argsort(distances)[:k]]
    counts = Counter(neighbor_labels)
    prediction = sorted(counts.items(), key = lambda item: item[1], reverse = True)[0][0]
    return(prediction)

def knn(X_train, X_test, y_train, y_test, k):
    '''
    make prediction for a single test data point given training data and k
    X_test and y_test are for single data points
    X_train and y_train are for entire training data set 
    k must be an integer
    '''
    distances = sort_distances(X_train, X_test)
    prediction = predict_class(distances, y_train, k)
    
    print('expected class: %s, npredicted class: %s' % (y_test, prediction))
    return((prediction, y_test))
    

Now we make a prediction for each data point in the test data set using the training data set.

In [78]:
knn_predictions = list()

for i in range(len(X_test)):
    result = knn(X_train, X_test[i], y_train, y_test[i], 5)
    knn_predictions.append(result)

expected class: 1, npredicted class: 1
expected class: 0, npredicted class: 0
expected class: 1, npredicted class: 1
expected class: 2, npredicted class: 2
expected class: 2, npredicted class: 2
expected class: 1, npredicted class: 1
expected class: 2, npredicted class: 2
expected class: 0, npredicted class: 0
expected class: 1, npredicted class: 1
expected class: 2, npredicted class: 2
expected class: 1, npredicted class: 1
expected class: 2, npredicted class: 2
expected class: 2, npredicted class: 2
expected class: 1, npredicted class: 1
expected class: 1, npredicted class: 1
expected class: 0, npredicted class: 0
expected class: 2, npredicted class: 2
expected class: 2, npredicted class: 2
expected class: 1, npredicted class: 1
expected class: 2, npredicted class: 2
expected class: 0, npredicted class: 0
expected class: 1, npredicted class: 2
expected class: 2, npredicted class: 2
expected class: 2, npredicted class: 2
expected class: 0, npredicted class: 0
expected class: 1, npredi

Calculate accuracy for predictions.

In [85]:
accuracy = sum([x[0]==x[1] for x in knn_predictions])/len(knn_predictions)*100
print("prediction accuracy on test set = %.2f%%" % accuracy)

prediction accuracy on test set = 97.37%


An alternative approach could have been to use k-fold cross validation and evaluate performance on each fold of data, e.g. 

```python
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
folds = kf.split(iris.data, iris.target)
```